In [1]:
pip install llama-index llama-index-embeddings-huggingface llama-index-llms-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.1/282.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [1]:
pip install llama-index-readers-file pymupdf

In [2]:
pip install sentence-transformers

In [3]:
import os
from google.colab import userdata

GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [4]:
%load_ext autoreload
%autoreload 2

In [8]:
from llama_index.core import SimpleDirectoryReader

docs = SimpleDirectoryReader("/content/data/marie curie docs/").load_data()
len(docs)

10

In [10]:
from llama_index.core import Document

doc_text = "\n\n".join([doc.get_content() for doc in docs])
docs = [Document(text = doc_text)]
len(docs)

1

In [11]:
### split the docs
from llama_index.core.node_parser import HierarchicalNodeParser, SentenceSplitter

node_parser = HierarchicalNodeParser.from_defaults()

nodes = node_parser.get_nodes_from_documents(docs)

len(nodes)

18

In [13]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

leaf = get_leaf_nodes(nodes)
root = get_root_nodes(nodes)

print(len(leaf))
print(len(root))

14
1


In [14]:
## Store then into Storage
from llama_index.core import StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

docstore = SimpleDocumentStore()

docstore.add_documents(nodes)

storage_context = StorageContext.from_defaults(docstore = docstore)

llm = Groq(model="gemma2-9b-it")

In [15]:
embedding = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(leaf, storage_context = storage_context, embed_model = embedding)

In [17]:
### create a retriever on top of index
from llama_index.core.retrievers import AutoMergingRetriever

base_retriever = index.as_retriever(similarity_top_k=6)

retriever = AutoMergingRetriever(
    base_retriever,
    storage_context,
    verbose=True,
)

In [18]:
query = (
    "What did Marie Curie win Nobel Prizes for?"
)
nodes = retriever.retrieve(query)
base_nodes = base_retriever.retrieve(query)

> Merging 3 nodes into parent node.
> Parent node id: a4ba8962-aa3e-4e2c-85da-97426f881e8c.
> Parent node text: Marie Curie's legacy revolutionized healthcare.
Her discoveries laid the groundwork for diagnosti...



In [19]:
print(len(nodes))
print(len(base_nodes))

4
6


In [20]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=10000)

**Node ID:** dac60e8e-dedd-4d1a-a5d5-70725b362afe<br>**Similarity:** 0.786636601368507<br>**Text:** She set a precedent for using scientific discovery to benefit humanity.
Curie's legacy lives on in hospitals and treatment centers worldwide.

Marie Curie received the Nobel Prize in Physics in 1903, shared with Pierre Curie and Henri Becquerel.
She became the first woman to ever win a Nobel Prize.
In 1911, she won the Nobel Prize in Chemistry for discovering radium and polonium.
She is the only person to have won Nobel Prizes in two scientific disciplines.
Her recognition helped highlight the importance of women in science.
Both awards acknowledged her tireless work on radioactivity and elemental discovery.<br>

**Node ID:** 060f1fae-59ee-4330-b7d8-192a061eeaa4<br>**Similarity:** 0.7177710862977854<br>**Text:** Both awards acknowledged her tireless work on radioactivity and elemental discovery.
Despite her fame, she remained modest and focused on her research.
Curie’s Nobel legacy inspired generations of scientists across the globe.
She donated prize money to research and relief efforts during wartime.
Her Nobel medals were hidden during WWII to protect them from the Nazis.

The use of radium in treating cancer marked the birth of radiotherapy.
Marie Curie’s discoveries showed that radioactive materials could target and kill cancerous tissue.
Radiotherapy became a foundation for oncology in the early 20th century.
Hospitals began to build specialized facilities to offer radiation treatments.<br>

**Node ID:** a4ba8962-aa3e-4e2c-85da-97426f881e8c<br>**Similarity:** 0.6186720746662985<br>**Text:** Marie Curie's legacy revolutionized healthcare.
Her discoveries laid the groundwork for diagnostic imaging and cancer treatment.
Hospitals worldwide now use radiation in life-saving procedures.
Curie’s mobile radiography units pioneered battlefield medicine.
Radioisotopes help detect organ function, cancers, and infections.
Radiation therapy remains one of the primary cancer treatments.
Her scientific integrity set high standards for medical research.
She emphasized accessibility to treatments regardless of wealth.
Modern nuclear medicine owes much to Curie’s early insights.
Her life’s work continues to benefit millions in the field of healthcare.

Marie Curie, born in 1867 in Warsaw, Poland, was a pioneering physicist and chemist. 
She moved to Paris to study at the Sorbonne and met her husband, Pierre Curie, there. 
Together, they conducted groundbreaking research on radioactivity. 
Marie was the first woman to win a Nobel Prize and the only person to win Nobel Prizes in two different scientific fields.
Her dedication to science paved the way for women in STEM. 
She named the element polonium after her homeland. 
Her early education took place under oppressive Russian rule, which suppressed Polish culture. 
Curie excelled in her studies despite poverty and adversity.
She later founded the Radium Institute in Paris. 
Marie Curie died in 1934 from aplastic anemia, likely caused by prolonged radiation exposure.

Marie and Pierre Curie's experiments led to the discovery of two new elements: polonium and radium. 
They identified that certain materials emitted a new kind of energy, which they called radioactivity. 
Their work built on Henri Becquerel's discovery of spontaneous radiation from uranium. 
Curie used painstaking methods to isolate radioactive isotopes from tons of pitchblende.
They discovered that radioactivity was a property of atoms themselves, not a chemical reaction.
This fundamentally changed the understanding of matter and atomic structure.
Their discoveries laid the foundation for nuclear physics and modern atomic theory.
Radium, in particular, emitted powerful radiation, which intrigued scientists globally.
Their work marked the beginning of a new era in physical science.
Marie Curie coined the term "radioactivity" to describe this phenomenon.

Radium’s intense radioactivity quickly found practical uses in medicine and industry.
Doctors began using radium to treat cancer through early forms of radiotherapy.
Radium was also used in luminous paints for watch dials and aircraft instruments.
Its glow-in-the-dark property made it popular, despite the health risks.<br>

**Node ID:** 5c96b2ad-bde5-41b2-bca1-d90f04dc65cf<br>**Similarity:** 0.5353331554970909<br>**Text:** Her handwriting reflects both clarity and intense focus.
The notes reveal the painstaking physical labor behind early radioactivity research.
Despite lacking modern lab tools, Curie achieved extraordinary accuracy.
The lab notebooks are historical artifacts and symbols of scientific perseverance.
They capture the daily challenges and triumphs of Curie’s groundbreaking work.

Curie’s discoveries redefined the field of chemistry.
She proved that atoms could emit energy, transforming the atomic model.
This challenged the long-standing belief that atoms were indivisible.
Chemistry shifted toward nuclear reactions and isotopic studies.
Radium's behavior opened new branches of inorganic chemistry.<br>

In [21]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

**Node ID:** dac60e8e-dedd-4d1a-a5d5-70725b362afe<br>**Similarity:** 0.786636601368507<br>**Text:** She set a precedent for using scientific discovery to benefit humanity.
Curie's legacy lives on in hospitals and treatment centers worldwide.

Marie Curie received the Nobel Prize in Physics in 1903, shared with Pierre Curie and Henri Becquerel.
She became the first woman to ever win a Nobel Prize.
In 1911, she won the Nobel Prize in Chemistry for discovering radium and polonium.
She is the only person to have won Nobel Prizes in two scientific disciplines.
Her recognition helped highlight the importance of women in science.
Both awards acknowledged her tireless work on radioactivity and elemental discovery.<br>

**Node ID:** a868f8f7-b04a-48ae-aaec-f81abd2c1f03<br>**Similarity:** 0.7357715010770798<br>**Text:** Her life’s work continues to benefit millions in the field of healthcare.

Marie Curie, born in 1867 in Warsaw, Poland, was a pioneering physicist and chemist. 
She moved to Paris to study at the Sorbonne and met her husband, Pierre Curie, there. 
Together, they conducted groundbreaking research on radioactivity. 
Marie was the first woman to win a Nobel Prize and the only person to win Nobel Prizes in two different scientific fields.
Her dedication to science paved the way for women in STEM. 
She named the element polonium after her homeland.<br>

**Node ID:** 060f1fae-59ee-4330-b7d8-192a061eeaa4<br>**Similarity:** 0.7177710862977854<br>**Text:** Both awards acknowledged her tireless work on radioactivity and elemental discovery.
Despite her fame, she remained modest and focused on her research.
Curie’s Nobel legacy inspired generations of scientists across the globe.
She donated prize money to research and relief efforts during wartime.
Her Nobel medals were hidden during WWII to protect them from the Nazis.

The use of radium in treating cancer marked the birth of radiotherapy.
Marie Curie’s discoveries showed that radioactive materials could target and kill cancerous tissue.
Radiotherapy became a foundation for oncology in the early 20th century.
Hospitals began to build specialized facilities to offer radiation treatments.<br>

**Node ID:** 29f4a822-b537-4438-ae56-5326c6f98542<br>**Similarity:** 0.5647762221502397<br>**Text:** She named the element polonium after her homeland. 
Her early education took place under oppressive Russian rule, which suppressed Polish culture. 
Curie excelled in her studies despite poverty and adversity.
She later founded the Radium Institute in Paris. 
Marie Curie died in 1934 from aplastic anemia, likely caused by prolonged radiation exposure.

Marie and Pierre Curie's experiments led to the discovery of two new elements: polonium and radium. 
They identified that certain materials emitted a new kind of energy, which they called radioactivity.<br>

**Node ID:** 203c4b9e-23bf-49c7-8fcb-35868b78ea10<br>**Similarity:** 0.555468500771576<br>**Text:** Marie Curie's legacy revolutionized healthcare.
Her discoveries laid the groundwork for diagnostic imaging and cancer treatment.
Hospitals worldwide now use radiation in life-saving procedures.
Curie’s mobile radiography units pioneered battlefield medicine.
Radioisotopes help detect organ function, cancers, and infections.
Radiation therapy remains one of the primary cancer treatments.
Her scientific integrity set high standards for medical research.
She emphasized accessibility to treatments regardless of wealth.
Modern nuclear medicine owes much to Curie’s early insights.
Her life’s work continues to benefit millions in the field of healthcare.<br>

**Node ID:** 5c96b2ad-bde5-41b2-bca1-d90f04dc65cf<br>**Similarity:** 0.5353331554970909<br>**Text:** Her handwriting reflects both clarity and intense focus.
The notes reveal the painstaking physical labor behind early radioactivity research.
Despite lacking modern lab tools, Curie achieved extraordinary accuracy.
The lab notebooks are historical artifacts and symbols of scientific perseverance.
They capture the daily challenges and triumphs of Curie’s groundbreaking work.

Curie’s discoveries redefined the field of chemistry.
She proved that atoms could emit energy, transforming the atomic model.
This challenged the long-standing belief that atoms were indivisible.
Chemistry shifted toward nuclear reactions and isotopic studies.
Radium's behavior opened new branches of inorganic chemistry.<br>

In [22]:
### Define The Query Engine
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)
base_query_engine = RetrieverQueryEngine.from_args(base_retriever, llm=llm)

In [23]:
response = query_engine.query(query)

str(response)

> Merging 3 nodes into parent node.
> Parent node id: a4ba8962-aa3e-4e2c-85da-97426f881e8c.
> Parent node text: Marie Curie's legacy revolutionized healthcare.
Her discoveries laid the groundwork for diagnosti...



'She won Nobel Prizes for her tireless work on radioactivity and elemental discovery.  She also won a Nobel Prize for discovering radium and polonium. \n'

In [24]:
base_response = base_query_engine.query(query)

str(base_response)

'She won Nobel Prizes for her tireless work on radioactivity and elemental discovery. \n'